In [1]:
#import libraries
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
#defining the variable time
time = 200
d_mu = 100
d_std = 25
d = np.maximum(np.random.normal(d_mu, d_std, time).round(0).astype(int),0)

In [3]:
#define the remaining inputs of our policy and compute the various parameters
L, R, alpha = 4, 1, 0.95 
z = norm.ppf(alpha) 
x_std = np.sqrt(L+R)*d_std
x_std

55.90169943749474

In [4]:
#Cycle stock Cs. Stock needed to fulfill the expected demand
#Safety stock Ss. Stock protecting the supply chain against demand variability
#In-transit stock Is. Inventory in transit in between different locations. Usually
Ss = np.round(x_std*z).astype(int) 
Cs = 1/2 * d_mu * R 
Is = d_mu * L 
S = Ss + 2*Cs + Is

In [5]:
#orders can stay in transit for 4 periods
hand = np.zeros(time, dtype=int) 
transit = np.zeros((time,L+1), dtype=int)

In [6]:
#We also have to create two objects to track the service level of our policy
stockout_period = np.full(time, False, dtype=bool)
stockout_cycle = []

In [7]:
#initialize these arrays for the first timestep
hand[0] = S - d[0]
transit[1,-1] = d[0]

In [8]:
#Making the simulation
for t in range(1,time): 
  if transit[t-1,0]>0: 
    stockout_cycle.append(stockout_period[t-1]) 
  hand[t] = hand[t-1] - d[t] + transit[t-1,0] 
  stockout_period[t] = hand[t] < 0
  transit[t,:-1] = transit[t-1,1:]
  if 0==t%R: 
    net = hand[t] + transit[t].sum() 
    transit[t,L] = S - net

In [44]:
#create a DataFrame df to hold our simulation results
df = pd.DataFrame(data= {'Demand':d, 'On−hand':hand, 'In−transit':list(transit)}) 
df = df.iloc[R+L:,:] #Remove initialization periods 
print(df)

     Demand  On−hand                In−transit
5       130     -172  [117, 253, 86, 159, 149]
6        34     -166  [253, 86, 159, 149, 111]
7       104      -30  [86, 159, 149, 111, 117]
8        74      -25  [159, 149, 111, 117, 81]
9        54       56   [149, 111, 117, 81, 78]
..      ...      ...                       ...
195     105      103   [125, 67, 108, 82, 107]
196     106      130    [67, 108, 82, 107, 98]
197      77       91   [108, 82, 107, 98, 106]
198      97      143    [82, 107, 98, 106, 56]
199     101      130    [107, 98, 106, 56, 95]

[195 rows x 3 columns]
